In [2]:
import h2o
import pandas as pd
import sys
from pysparkling import *
h2o.__version__
hc = H2OContext(sc).start()
print(hc)

ImportError: No module named pysparkling

In [2]:
faults = h2o.import_file(path="hdfs://dscluster/user/212362409/gewgr_fault_d.csv",header=1)
fault_data = h2o.import_file(path="hdfs://dscluster/user/212362409/gewgr_fault_session_f.txt",header=1)
channels = h2o.import_file(path="hdfs://dscluster/user/212362409/channel_d.csv",header=1)
channel_data = h2o.import_file(path="hdfs://dscluster/user/212362409/channel_data_f_new3.txt",header=1)
turbines = h2o.import_file(path="hdfs://dscluster/user/212362409/turbine.csv",header=1)


Parse Progress: [##################################################] 100%

Parse Progress: [##################################################] 100%

Parse Progress: [##################################################] 100%

Parse Progress: [##################################################] 100%

Parse Progress: [##################################################] 100%


In [3]:
tcd = channel_data[(channel_data['turbine_id']==38219) | (channel_data['turbine_id']==38228) | (channel_data['turbine_id']==38231) 
            | (channel_data['turbine_id']==38232) | (channel_data['turbine_id']==38251) | (channel_data['turbine_id']==38255)]
tcd.dim

[5548122, 12]

In [4]:
tfd = fault_data[(fault_data['turbine_id']==38219) | (fault_data['turbine_id']==38228) | (fault_data['turbine_id']==38231) 
            | (fault_data['turbine_id']==38232) | (fault_data['turbine_id']==38251) | (fault_data['turbine_id']==38255)]
tfd.dim

[328589, 25]

In [5]:
tfduse = tfd[tfd['fault_session_start_time'] >= tcd['local_dttm'].min()]
tfduse.dim

[6039, 25]

In [6]:
grpd = tfduse.group_by(by=['fault_dim_key'])
grpd.count()
tr = grpd.get_frame()
trpd = h2o.as_list(tr)
trpd

,fault_dim_key,nrow_fault_session_fct_key
0,3,2
1,9,4
2,14,3
3,28,3
4,47,1
5,52,4
6,63,2
7,77,22
8,105,1
9,124,1


In [7]:
channels = tcd['channel_id'].unique()
fdata = h2o.as_list(tfduse)
channelsused = h2o.as_list(channels)
channelsused = channelsused['C1'].tolist()
channelsused.sort()
print channelsused

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 25, 26, 28, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 44, 54, 69, 70, 71, 78, 269, 270, 271, 323, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 361, 366, 367, 368, 369, 370, 371, 372, 373, 374, 376, 377, 378, 379, 380, 381, 382, 390, 391, 392, 393, 394, 395, 625, 626, 629, 631, 641, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 668, 677, 678, 680, 681, 682, 683, 685, 691, 692, 693, 694, 695, 696, 697, 712, 718, 719, 726, 742, 743, 754, 756]


In [1]:
cduse = tcd[tcd['channel_id'].isin(channelsused)]
cduse.describe()

NameError: name 'tcd' is not defined

In [61]:
import numpy as np
temprow = []

for i in range(0,len(fdata)): 
    print("i",i)
    tm = fdata.fault_session_start_time[i]
    cdblock = cduse[(cduse['local_dttm'] <= tm) & (cduse['local_dttm'] >= tm - 6e5)
                   & (cduse['turbine_id'] == fdata.turbine_id[i])]
                   #tm-1.8e6)] #30 min
    cdpd = h2o.as_list(cdblock)
    cdpd = cdpd.sort_values(by='channel_id', ascending=True)
    cdpd = cdpd.reset_index(drop=True)
    cid = -1
    channelcounter=0
    temp=[]
    #;cname = []
    #print cdpd
    
    for j in range(0,len(cdpd)):
        if j==0:
            #print (j,cdpd.channel_id[j],channelsused[0])
            if (cdpd.channel_id[j] == channelsused[0]):
                temp.append(cdpd.iloc[j,7:11].values.tolist())
            else:
                temp.append([np.nan,np.nan,np.nan,np.nan])
            cid = cdpd.channel_id[j]
            channelcounter = channelcounter+1
            #cname.append(cid)
            #print cid
        else:
            if cdpd.channel_id[j] != cid:
                #print("j",j," channlecntr:",channelcounter)
                #print (j,cdpd.channel_id[j],channelsused[channelcounter])
                if (cdpd.channel_id[j] == channelsused[channelcounter]):
                    temp.append(cdpd.iloc[j,7:11].values.tolist())
                    cid = cdpd.channel_id[j]
                else:
                    temp.append([np.nan,np.nan,np.nan,np.nan])
                    cid = channelsused[channelcounter] #wrote this channel                  
               
                channelcounter = channelcounter+1
                #if channelcounter > 135:
                #    break
                    
    ary = np.array(temp)
    #print ary.shape    
    arow = list(ary.ravel())
    temprow.append(arow)
   
x = pd.DataFrame(temprow)
x.shape

('i', 0)
('i', 1)
('i', 2)
('i', 3)
('i', 4)
('i', 5)
('i', 6)
('i', 7)
('i', 8)
('i', 9)
('i', 10)
('i', 11)
('i', 12)
('i', 13)
('i', 14)
('i', 15)
('i', 16)
('i', 17)
('i', 18)
('i', 19)
('i', 20)
('i', 21)
('i', 22)
('i', 23)
('i', 24)
('i', 25)
('i', 26)
('i', 27)
('i', 28)
('i', 29)
('i', 30)
('i', 31)
('i', 32)
('i', 33)
('i', 34)
('i', 35)
('i', 36)
('i', 37)
('i', 38)
('i', 39)
('i', 40)
('i', 41)
('i', 42)
('i', 43)
('i', 44)
('i', 45)
('i', 46)
('i', 47)
('i', 48)
('i', 49)
('i', 50)
('i', 51)
('i', 52)
('i', 53)
('i', 54)
('i', 55)
('i', 56)
('i', 57)
('i', 58)
('i', 59)
('i', 60)
('i', 61)
('i', 62)
('i', 63)
('i', 64)
('i', 65)
('i', 66)
('i', 67)
('i', 68)
('i', 69)
('i', 70)
('i', 71)
('i', 72)
('i', 73)
('i', 74)
('i', 75)
('i', 76)
('i', 77)
('i', 78)
('i', 79)
('i', 80)
('i', 81)
('i', 82)
('i', 83)
('i', 84)
('i', 85)
('i', 86)
('i', 87)
('i', 88)
('i', 89)
('i', 90)
('i', 91)
('i', 92)
('i', 93)
('i', 94)
('i', 95)
('i', 96)
('i', 97)
('i', 98)
('i', 99)
('i', 100)

(6039, 544)

In [62]:
x.to_csv('turbinesMat.csv')

In [9]:
xmat = pd.read_csv('turbinesMat.csv')
xmat.drop(xmat.columns[0],axis=1, inplace=True)
xmat.shape

(6039, 544)

In [10]:
xmat.drop(xmat.columns[xmat.apply(lambda col: col.isnull().sum() > 1500)], axis=1, inplace=True)
xmat.shape

(6039, 392)

In [11]:
ycol = fdata['fault_dim_key'].map(lambda x: 1 if x==77 else 0)
#print ycol[ycol==1]
xmat =  pd.concat([xmat, ycol], axis=1)
xmat.shape

(6039, 393)

In [12]:
hmat = h2o.H2OFrame.from_python(xmat, destination_frame="hmat")


Parse Progress: [##################################################] 100%


In [13]:
hmat.impute(column=-1)

[286.39392743528754,
 75.2478448609279,
 468.1144915742469,
 140.54532822303014,
 2786.8665201145927,
 0.03323727670247573,
 2787.2873836403246,
 2786.844523668053,
 0.528089730199266,
 0.8858328911544595,
 0.5574979955940662,
 -0.13812290078591227,
 31.145490155896507,
 23.593511779100044,
 82.8163592591077,
 -21.199108270240913,
 400.1764522075819,
 1.1592952713567823,
 402.8073063965735,
 397.6095530101337,
 400.78003399327986,
 1.1539180454822573,
 403.38483926247994,
 398.1915548244936,
 402.2962133506189,
 1.1512599960013616,
 404.9039819981402,
 399.741702107925,
 265.93518706139355,
 68.90478149865486,
 441.0395159836634,
 144.85082251587866,
 249.35889041974198,
 65.00565426748247,
 415.865563021855,
 137.32638277524953,
 252.56046737211437,
 66.17237251390547,
 422.3816213722719,
 137.85688153495062,
 546.8915434399509,
 77.8415299191569,
 678.5527741447376,
 412.7562878575012,
 6.3641664103322695,
 0.8937462005538923,
 7.867230300455453,
 4.7993164184801245,
 4.4195504322029

In [3]:
hmat.describe

NameError: name 'hmat' is not defined

In [15]:
hmat['C393']  = hmat['C393'].asfactor() 

In [16]:
[train,test] = hmat.split_frame(ratios=[0.8],destination_frames=["train", "test"], seed=100)
xcols = hmat.columns

In [17]:
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch

rfTurbines = H2ORandomForestEstimator(model_id="rfTurbines", seed=12, ntrees=20, max_depth=20, balance_classes=False)
rfTurbines.train(x=xcols[0:391], y=xcols[392], training_frame=train, validation_frame=test)

h2o.download_pojo(rfTurbines, path=".")


drf Model Build Progress: [##################################################] 100%
Filepath: ./rfTurbines.java


In [26]:
#test.describe()
#testpd = h2o.download_csv(test,"./testMat.csv")

In [18]:
out = rfTurbines.predict(test)


drf prediction Progress: [##################################################] 100%


In [22]:
out.describe()

Rows:1,201 Cols:3

Chunk compression summary: 


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C0L,Constant Integers,8,33.333336,640 B,14.253898
CX0,Zero Sparse Bits,2,8.333334,154 B,3.4298441
CXD,Zero Sparse Reals,7,29.166666,718 B,15.991092
CUD,Unique Reals,7,29.166666,2.9 KB,66.32517



Frame distribution summary: 


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
10.42.50.155:54325,2.6 KB,715.0,5.0,15.0
10.42.50.156:54321,1.8 KB,486.0,3.0,9.0
mean,2.2 KB,600.5,4.0,12.0
min,1.8 KB,486.0,3.0,9.0
max,2.6 KB,715.0,5.0,15.0
stddev,451 B,114.5,1.0,3.0
total,4.4 KB,1201.0,8.0,24.0


,predict,p0,p1
type,enum,real,real
mins,0.0,0.2,0.0
mean,0.00249791840133,0.997044129892,0.00295587010824
maxs,1.0,1.0,0.8
sigma,0.0499375129937,0.0384274929913,0.0384274929913
zeros,1198,0,1181
missing,0,0,0
0,0,1.0,0.0
1,0,0.5,0.5
2,0,1.0,0.0


In [23]:
h2o.download_csv(test,"./testData.csv")

In [24]:
h2o.download_csv(out,"./outClassification.csv")

In [33]:
hmatReal = hmat
hmatReal['C393']  = hmatReal['C393'].asnumeric()
[trainReal,testReal] = hmatReal.split_frame(ratios=[0.8],destination_frames=["trainReal", "testReal"], seed=100)
xcolsReal = hmatReal.columns
rfTurbinesReal = H2ORandomForestEstimator(model_id="rfTurbinesReal", seed=12, ntrees=20, max_depth=20, balance_classes=False)
rfTurbinesReal.train(x=xcolsReal[0:391], y=xcolsReal[392], training_frame=trainReal, validation_frame=testReal)


drf Model Build Progress: [##################################################] 100%


In [34]:
outReal= rfTurbinesReal.predict(testReal)
outReal.describe()


drf prediction Progress: [##################################################] 100%
Rows:1,201 Cols:1

Chunk compression summary: 


chunk_type,chunk_name,count,count_percentage,size,size_percentage
C0L,Constant Integers,4,50.0,320 B,46.64723
CXD,Zero Sparse Reals,4,50.0,366 B,53.352768



Frame distribution summary: 


,size,number_of_rows,number_of_chunks_per_column,number_of_chunks
10.42.50.155:54325,428 B,715.0,5.0,5.0
10.42.50.156:54321,258 B,486.0,3.0,3.0
mean,343 B,600.5,4.0,4.0
min,258 B,486.0,3.0,3.0
max,428 B,715.0,5.0,5.0
stddev,85 B,114.5,1.0,1.0
total,686 B,1201.0,8.0,8.0


,predict
type,real
mins,0.0
mean,0.00308076602831
maxs,1.0
sigma,0.0501547701761
zeros,1194
missing,0
0,0.0
1,0.5
2,0.0


In [35]:
h2o.download_csv(outReal,"./outRealClassification.csv")

In [18]:
# Grid search
ntrees_opt = [10, 20, 30]
max_depth_opt = [10, 15, 25]
hyper_parameters = {"ntrees": ntrees_opt, "max_depth": max_depth_opt}

rf_grid = H2OGridSearch(H2ORandomForestEstimator, hyper_params=hyper_parameters)

rf_grid.train(x=xcols[0:391], y=xcols[392], training_frame=train, validation_frame=test)


drf Grid Build Progress: [##################################################] 100%


In [28]:
#h2o.download_csv(hmat,"./hmat.csv")